In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

from sys import path
from os.path import abspath
path.append(abspath("/home/ng136/nico"))
import ng_functions as ng

from glob import glob
from pathlib import Path
import warnings
from time import time
from datetime import datetime
from tqdm import tqdm

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import ConfusionMatrixDisplay

# I/O

# Load annotated experiments 2 and 3

In [3]:
n_states = ['N1a', 'N1b', 'N2', 'N3', 'N4', 'N5', 'N6']


In [4]:
n_states_dict = {'N1a' : 'Sell+Ngp+ neutrophils',
                 'N1b' : 'Sell+Lst1+ neutrophils',
                 'N2' : 'Sell+Cxcl10+ neutrophils',
                 'N3' : 'Cxcl3+ neutrophils',
                 'N4' : 'Siglecf+Xbp1+ neutrophils',
                 'N5' : 'Siglecf+Ccl3+ neutrophils',
                 'N6' : 'Siglecf+Ngp+ neutrophils'}

neu_palette_dict = {'Sell+Ngp+ neutrophils': '#6EA6CD',
                    'Sell+Lst1+ neutrophils': '#4A7BB7',
                    'Sell+Cxcl10+ neutrophils': '#364B9A',
                    'Cxcl3+ neutrophils': '#EAECCC',
                    'Siglecf+Xbp1+ neutrophils': '#A50026',
                    'Siglecf+Ccl3+ neutrophils': '#DD3D2D',
                    'Siglecf+Ngp+ neutrophils': '#F67E4B'}

# GEX

In [6]:
ndata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_all_cells_untreated_annotated_embedding_8404x13126_backup_220325_11h55.h5ad')
tdata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_all_cells_acd40_annotated_embedding_33603x17824_backup_220325_12h01.h5ad')

ndata.obs['cell_state_annotation'] = ndata.obs['smoothed_Zilionis']
tdata.obs['cell_state_annotation'] = tdata.obs['smoothed_Zilionis']

hdata = ndata[ndata.obs.sample_condition.str.startswith('WT')].copy()
udata = ndata[ndata.obs.sample_condition.str.startswith('KP19')].copy()


# ADT

In [9]:
nadtdata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_all_cells_untreated_adt_clr_no_cd101_6719x20_backup_221215_14h21.h5ad')
tadtdata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_all_cells_treated_adt_clr_33603x20_backup_221215_14h27.h5ad')

nadtdata.obs['cell_state_annotation'] = nadtdata.obs['smoothed_Zilionis']
tadtdata.obs['cell_state_annotation'] = tadtdata.obs['smoothed_Zilionis']

hadtdata = nadtdata[nadtdata.obs.sample_condition.str.startswith('WT')].copy()
uadtdata = nadtdata[nadtdata.obs.sample_condition.str.startswith('KP19')].copy()

# SPRING

# Set up X 

In [10]:
ndata.X = ndata.layers['cp10k']
hdata.X = hdata.layers['cp10k']
udata.X = udata.layers['cp10k']
tdata.X = tdata.layers['cp10k']

nadtdata.X = nadtdata.layers['clr']
hadtdata.X = hadtdata.layers['clr']
uadtdata.X = uadtdata.layers['clr']
tadtdata.X = tadtdata.layers['clr']

# Set up obs

In [11]:
for adata in [ndata, hdata, udata, tdata]:
    adata.obs = adata.obs.loc[:,adata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
                                                        'library_name', 'sample_condition', 
                                                        'cell_state_annotation'])]
    
    
for adtdata in [nadtdata, hadtdata, uadtdata]:
    adtdata.obs = adtdata.obs.loc[:,adtdata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
                                                    'library_name', 'sample_condition', 'CD101_Ab', 'Lair1_Ab',
                                                    'cell_state_annotation'])]


tadtdata.obs = tadtdata.obs.loc[:,tadtdata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
                                                    'library_name', 'sample_condition', 'Rat_IgG2a_Ab', 'cell_state_annotation'])]
    

# SAVE SPRING

In [15]:
ng.save_spring(ndata, '/n/groups/klein/nico/neutrophils/spring/221215', sample_name='all_cells_untreated_gex', embedding_name='gex')
ng.save_spring(hdata, '/n/groups/klein/nico/neutrophils/spring/221215', sample_name='all_cells_healthy_gex', embedding_name='gex')
ng.save_spring(udata, '/n/groups/klein/nico/neutrophils/spring/221215', sample_name='all_cells_kp19_gex', embedding_name='gex')
ng.save_spring(tdata, '/n/groups/klein/nico/neutrophils/spring/221215', sample_name='all_cells_acd40_gex', embedding_name='gex')

Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_untreated_gex/gex


/n/groups/klein/nico/neutrophils/analysis/conda-env/lib/python3.8/site-packages/scanpy/external/exporting.py:151: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(adata.obs[obs_name]):
/n/groups/klein/nico/neutrophils/analysis/conda-env/lib/python3.8/site-packages/scanpy/external/exporting.py:167: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(adata.obs[obs_name]):


Elapsed time: 0 minutes and 26.5 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_healthy_gex/gex
Elapsed time: 0 minutes and 17.9 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_kp19_gex/gex
Elapsed time: 0 minutes and 19.7 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_acd40_gex/gex
Elapsed time: 1 minutes and 32.5 seconds.


In [17]:
ng.save_spring(nadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='all_cells_untreated_adt', embedding_name='adt')
ng.save_spring(hadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='all_cells_healthy_adt', embedding_name='adt')
ng.save_spring(uadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='all_cells_kp19_adt', embedding_name='adt')
ng.save_spring(tadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='all_cells_acd40_adt', embedding_name='adt')

Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_untreated_adt/adt
Elapsed time: 0 minutes and 6.0 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_healthy_adt/adt
Elapsed time: 0 minutes and 3.3 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_kp19_adt/adt
Elapsed time: 0 minutes and 2.8 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/all_cells_acd40_adt/adt
Elapsed time: 0 minutes and 35.3 seconds.


# Copy to SPRING directory

In [18]:
import shutil
import glob

In [21]:
spring_projects = glob.glob('/n/groups/klein/nico/neutrophils/spring/221215/all_cells_*')

In [22]:
spring_projects

['/n/groups/klein/nico/neutrophils/spring/all_cells_kp19_gex',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_untreated_gex',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_acd40_adt',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_kp19_adt',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_untreated_adt',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_acd40_gex',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_healthy_gex',
 '/n/groups/klein/nico/neutrophils/spring/all_cells_healthy_adt']

In [25]:
for spring in spring_projects:
    shutil.copytree(spring, '/n/groups/kleintools/SPRING_private/gungabeesoon22/' + spring.split('/')[-1])
